<a href="https://colab.research.google.com/github/Ali-mohammadi-design/EBD-TRIZ-LLM/blob/main/EBD_TRIZ_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade pydantic==1.10.9

In [2]:
!pip install langchain==0.0.307
!pip install openai==0.27.0
!pip install tiktoken==0.3.0
!pip install wikipedia
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate, HumanMessagePromptTemplate, AIMessagePromptTemplate, SystemMessagePromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain, SequentialChain
from langchain.chains.openai_functions import create_structured_output_chain
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.chains.openai_functions import create_structured_output_chain
from langchain.text_splitter import CharacterTextSplitter

In [4]:
import os
from langchain.chat_models import ChatOpenAI

os.environ["OPENAI_API_KEY"] =  "YOUR OPOEN AI API KEY"

llm = ChatOpenAI(temperature=0)


**Step1: Asking Questions**

In [5]:
json_schema={'title':'questions related to the verbs',
             'description':'questions about the verbs of the sentences',
             'type':'object',
             'properties':{
                 'question with the why': {'title':'question with the why','description':'ask why from verb of the sentence','data_type':'string'},
                 'question with the where': {'title':'question with the where','description':'ask where from verb of the sentence','data_type':'string'},
                 'question with the when': {'title':'question with the when','description':'ask when from verb of the sentence','data_type':'string'},
                 'question with the how': {'title':'question with the how','description':'ask how from verb of the sentence','data_type':'string'},
                 'question with the who': {'title':'question with the who','description':'ask who from verb of the sentence','data_type':'string'}

             },

             'required':['question with the why','question with the where','question with the when','question with the how']
             }


In [6]:
def question_asking(sentence):
  #First section: Generating questions for the nouns:

  #In the first part of the process, the function extracts the nouns from the input text.
  system_template1=" You are an AI assistant that gets a text and returns all nouns, pronouns, and noun phrases. Please recognize verbs but do not return verbs or propositions"
  system_message_prompt1= SystemMessagePromptTemplate.from_template(system_template1)
  human_template1='please list all the pronouns, nouns, and noun phrases  in the following text: \n {text}'
  human_message_prompt1=HumanMessagePromptTemplate.from_template(human_template1)
  prompt1=ChatPromptTemplate.from_messages([system_message_prompt1,human_message_prompt1])
  chain1=LLMChain(llm=llm, prompt=prompt1, output_key='nouns')

#In the following section, we will determine whether this noun refers to a human or a non-human entity.
  human_template2='Consider the following list of nouns,pronouns,and nouns phrase. please specify which ones are human and which ones are non-human.This is the list: {nouns}'
  prompt2=ChatPromptTemplate.from_template(human_template2)
  chain2=LLMChain(llm=llm, prompt=prompt2, output_key='type')

# The following code generates questions with 'what' or 'who' for every noun.
  system_template3=''' you are an AI assistant that get nouns,pronouns,and nouns phrases and  their type and ask question with what or who.
  for the human nouns,pronouns,and nouns-phrases you ask: who is + (nouns,pronouns,and nouns phrases)? \n For non-human ones, you ask: what is + (nouns,pronouns,and nouns phrases)?'''
  system_message_prompt3= SystemMessagePromptTemplate.from_template(system_template3)
  human_template3='''please consider the folowing nouns. for each noun you must ask questions with what or who depends on their type.
  for example if the nouns are teacher and sandwich you shoud return who is teacher? and what is sandwich? The nouns are : \n {type}'''
  human_message_prompt3=HumanMessagePromptTemplate.from_template(human_template3)
  noun1='Bob, school, sandwich'
  example_one= HumanMessagePromptTemplate.from_template(noun1)
  question1='who is Bob? \n what is school? \n what is sandwich?'
  example_one_output= AIMessagePromptTemplate.from_template(question1)
  prompt3=ChatPromptTemplate.from_messages([system_message_prompt3,human_message_prompt3])
  chain3=LLMChain(llm=llm, prompt=prompt3, output_key='questions')

#Executing the chain
  text=sentence
  seq_chain=SequentialChain(chains=[chain1,chain2,chain3], input_variables=['text'],output_variables=['nouns','type','questions'],verbose=False)
  result1=seq_chain(text)
  noun_questions=result1


#Second section: Generating questions for the verbs

# The first part of this section would separate the text into sentences.
  system_template_verb_0=''' "You are an AI assistant who aims to get a text and separate its sentences.
  '''
  system_message_prompt0= SystemMessagePromptTemplate.from_template(system_template_verb_0)
  human_template_verb_0='''' "I'll provide a text for you. Please examine the given text, first identifying its verb phrases.
  After that, take note of each verb phrase you've identified in the provided text and return separate sentences for each verb phrase that include subject + verb phrase + object.
  \n The sentence is: \n {text}"
  '''
  human_message_prompt0=HumanMessagePromptTemplate.from_template(human_template_verb_0)
  prompt_verb_0=ChatPromptTemplate.from_messages([system_message_prompt0,human_message_prompt0])
  chain_verb_0=LLMChain(llm=llm, prompt=prompt_verb_0, output_key='all_sentences')

# The second part of this section would consider each sentence and generate various questions with 4 wh questions and with each separate sentence.
  system_template_verb_2=''' You are an AI assistant who aims to help designers to ask question.
  '''
  system_message_prompt_verb_2= SystemMessagePromptTemplate.from_template(system_template_verb_2)
  human_template_verb_2=''''
  I will provide you with a text containing subjects, verbs, objects and the sentences. Your task is to ask questions.
  generate five questions for each sentence using 'where, why, how, when, who.
  You must follow this structure:'wh question' + 'auxiliary verb' + 'subject' + 'verb' + 'object' + '?'.
  This means that for each sentence, you will create one question for each of the five question words.
  \n\nThe text to be used for this exercise is:\n\n {all_sentences}"
  '''
  human_message_prompt_verb_2=HumanMessagePromptTemplate.from_template(human_template_verb_2)
  prompt_verb_2=ChatPromptTemplate.from_messages([system_message_prompt_verb_2,human_message_prompt_verb_2])
  chain_verb_2=LLMChain(llm=llm, prompt=prompt_verb_2, output_key='all_questions')

#Executing the chain of this section
  seq_chain2=SequentialChain(chains=[chain_verb_0,chain_verb_2], input_variables=['text'],output_variables=['all_sentences','all_questions'],verbose=False)
  result2=seq_chain2(text)
  verb_questions=result2

# The text will be modified to remove extra explanation and only include questions.
  template_pars_out= "Please return only the questions from the following passage. \n\n this is the text:\n\n {undesired_output}"
  chat_prompt_pars_out_template=ChatPromptTemplate.from_template(template_pars_out)
  chat_prompt=ChatPromptTemplate.from_messages([chat_prompt_pars_out_template])
  request_parsout=chat_prompt.format_prompt(undesired_output=verb_questions['all_questions']).to_messages()
  result_pars_out_final=llm(request_parsout)


#Third Section: Generating questions related to the meaning of verbs.

#Extracting the verbs
  system_template_verb_meaning_0=" You are an AI assistant that gets text. You get some text. first recoginize all the sentences in the text. Then for each sentence return the verb"
  system_message_prompt_verb_meaning_0= SystemMessagePromptTemplate.from_template(system_template_verb_meaning_0)
  human_template_verb_meaning_0='please first consider the sentences in this text and then list the vebr in each sentence: \n {all_sentences}'
  human_message_prompt_verb_meaning_0=HumanMessagePromptTemplate.from_template(human_template_verb_meaning_0)
  prompt_verb_meaning_0=ChatPromptTemplate.from_messages([system_message_prompt_verb_meaning_0,human_message_prompt_verb_meaning_0])
  chain_verb_meaning_0=LLMChain(llm=llm, prompt=prompt_verb_meaning_0, output_key='verb_list')

#Asking question with "what do you mean by + the verb?" template.
  system_template_verb_meaning_1=" you are an AI assistant that get verbs and ask question with what.you ask: what do you mean by + the verb?"
  system_message_prompt_verb_meaning_1= SystemMessagePromptTemplate.from_template(system_template_verb_meaning_1)
  human_template_verb_meaning_1='''please consider the folowing verbs. for each verb you must ask questions with what.
  for example, if the verb is eat you should return: what do you mean by eating? \n\n list of verbs are: {verb_list}'''
  human_message_prompt_verb_meaning_1=HumanMessagePromptTemplate.from_template(human_template_verb_meaning_1)
  prompt_verb_meaning_1=ChatPromptTemplate.from_messages([system_message_prompt_verb_meaning_1,human_message_prompt_verb_meaning_1])
  chain_verb_meaning_1=LLMChain(llm=llm, prompt=prompt_verb_meaning_1, output_key='verb_meaning_questions')

#Executing the chain
  text=sentence
  seq_chain3=SequentialChain(chains=[chain_verb_meaning_0,chain_verb_meaning_1], input_variables=['all_sentences'],output_variables=['verb_list','verb_meaning_questions'],verbose=False)
  result3=seq_chain3(text)
  verbs_meaning=result3


#Fourth section: Asking question from the whole design problem

  system_template0=''' You are an AI assistant who aims to help designers to extract as much information as possible from a design problem.
  You get a design problem.
  You should ask six questions with the following format:
  1- what do you mean by + the whole design problem +?
  2- Where do you need + the whole design problem + ?
  3- when do you need + the whole design problem + ?
  4- why do you need + the whole design problem + ?
  5- How do you need + the whole design problem + ?
  6- who is responsible for + the whole design problem + ?

  '''
  system_message_prompt0= SystemMessagePromptTemplate.from_template(system_template0)
  human_template0=''''
  \n The design problem is: \n {text}"
  '''
  human_message_prompt0=HumanMessagePromptTemplate.from_template(human_template0)
  prompt0=ChatPromptTemplate.from_messages([system_message_prompt0,human_message_prompt0])
  request=prompt0.format_prompt(text=text).to_messages()
  result0=llm(request)
  verb_questions2=result0


#Collecting all the generated questions as a list in a text string
  question_final_list= f"\n \n {noun_questions['questions']}   \n\n{verbs_meaning['verb_meaning_questions']}  \n \n{ result_pars_out_final.content}\n\n {verb_questions2.content} "

#Fifth section: Eliminating duplicated questions and providing  unique questions

  system_template0=''' You are an AI assistant who can recognize repetetive concepts.
  You will receive a list of questions, some of these questions may be repetitive or asking about similar concept.
  Your objective is to provide a list of unique questions without modifying the wording of the original questions or making any other changes to them.

  '''
  system_message_prompt0= SystemMessagePromptTemplate.from_template(system_template0)
  human_template0=''''
  \n This is the list of questions: \n {question_final_list}"
  '''
  human_message_prompt0=HumanMessagePromptTemplate.from_template(human_template0)
  prompt0=ChatPromptTemplate.from_messages([system_message_prompt0,human_message_prompt0])
  request=prompt0.format_prompt(question_final_list=question_final_list).to_messages()
  result0=llm(request)
  question_final_list_unique=result0

#printing unique questions
  return print(f"these are the questions should be answered:\n\n{question_final_list_unique.content} \n\n\n...........\n\n The preliminary questions:\n\n{question_final_list}")


In [7]:
question_asking('Design a house that can fly')

these are the questions should be answered:

1. What is a house?
2. What do you mean by designing?
3. Where did you design a house?
4. Why did you design a house?
5. How did you design a house?
6. When did you design a house?
7. Who designed a house?
8. Where can a house fly?
9. Why can a house fly?
10. How can a house fly?
11. When can a house fly?
12. Who can fly a house?
13. What do you mean by "design a house that can fly"?
14. Where do you need a house that can fly?
15. When do you need a house that can fly?
16. Why do you need a house that can fly?
17. How do you need a house that can fly?
18. Who is responsible for designing a house that can fly? 


...........

 The preliminary questions:


 
 What is a house?
What is a house?   

What do you mean by designing?  
 
1. Where did you design a house?
2. Why did you design a house?
3. How did you design a house?
4. When did you design a house?
5. Who designed a house?
6. Where can a house fly?
7. Why can a house fly?
8. How can a h

**Second Stage: Answering the questions**

In [8]:
# We need to create a class that can save stages of the lifecycle.
class object_lifecycle():
  def __init__(self,First_stage,Second_stage,Third_stage,Fourth_stage):
     self.first_stage =First_stage
     self.Second_stage=Second_stage
     self.Third_stage=Third_stage
     self.Fourth_stage=Fourth_stage

In [9]:
#We need GPT to provide the life cycle of objects in 4 stages, in json format without extra explanation, for efficient environment generation.
json_schema={'title':'scientist',
             'description':'four stages of the creation of the object',
             'type':'object',
             'properties':{
                 'first_stage': {'title':'first_stage','description':'The First stage of the lifecycle of this object','data_type':'string'},
                 'second_stage': {'title':'Second_stage','description':'The second stage of the lifecycle of this object','data_type':'string'},
                 'third_stage': {'title':'third_stage','description':'The third stage of the lifecycle of this object','data_type':'string'},
                 'last_stage': {'title':'last_stage','description':'The last stage of the lifecycle of this object','data_type':'string'},
             },

             'required':['first_stage','second_stage','thrid stage','last stage']
             }

In [10]:
def What_OR_Who_is(name):


#Clarifying the meaning of the object
  system_template=" you are a dictionary that could help designers to define {objects}. You would define {objects} clearly and briefly in one sentence"
  system_message_prompt= SystemMessagePromptTemplate.from_template(system_template)
  human_template= ' What is {objects}?'
  human_prompt= HumanMessagePromptTemplate.from_template(human_template)
  chat_prompt=ChatPromptTemplate.from_messages([system_message_prompt, human_prompt])
  request=chat_prompt.format_prompt(objects=name).to_messages()
  definition=llm(request)


# Recognizing the object's lifecycle, in the next ection we will generate objects in the environment of each stage using JSON format output in this section.
  system_template=" you are an AI assistant that could help designers to understand the lifecycle of {objects}. You would explain the lifecycle of {objects} clearly and very breifly in just four stages and provide a  short explanation for every stage "
  system_message_prompt= SystemMessagePromptTemplate.from_template(system_template)
  human_template= 'I would define lifecycle as events that something passes to reach the point that it is. For instance the lifecycle of an engineer is studying in school, getting good marks, going to colledge, going to university, finding an engineer job. What is life cycle of {objects}'
  human_prompt= HumanMessagePromptTemplate.from_template(human_template)
  chat_prompt=ChatPromptTemplate.from_messages([system_message_prompt, human_prompt])
  chain=create_structured_output_chain(json_schema,llm, chat_prompt,verbose=False)
  result=chain.run(objects=name)
  obj=object_lifecycle(result['first_stage'],result['second_stage'],result['third_stage'],result['last_stage'])

#Identifying the objects at every stage of their lifecycle
  system_template=''' Every {objects} is related to it's environment.
  we would have three types of environment: 1- built environment, 2-human environment 3- nature environment.
  Built environment: Every thing directly related to the {objects} and made by human.
  Human environment: Every human being directly related to the {objects}.
  Nature environment: Every natural things like weather, water etc that is directly connect to {objects}.
  When we wnat to describe the environment of {objects} we have to show the relation between {objects} and the environment in senctnces.
  you are an AI assistant that could help designers to understand the environment of {objects}.
  You would explain the environment of {objects} with single sentences.You would prefer to use action verbs in the sentences'''
  system_message_prompt= SystemMessagePromptTemplate.from_template(system_template)
  human_template= '''{lifecycle_stage} is one of the stages of lifecycle of {objects}.
   Consider {objects} in this stage and determine every part of the environment of {objects} in the separate sentences'''
  human_prompt= HumanMessagePromptTemplate.from_template(human_template)
  chat_prompt=ChatPromptTemplate.from_messages([system_message_prompt, human_prompt])
  model_request= chat_prompt.format_prompt(lifecycle_stage=obj.first_stage, objects=name).to_messages()
  env_in_first_stage=model(model_request)

  model_request2= chat_prompt.format_prompt(lifecycle_stage=obj.Second_stage, objects=name).to_messages()
  env_in_second_stage=model(model_request2)

  model_request3= chat_prompt.format_prompt(lifecycle_stage=obj.Third_stage, objects=name).to_messages()
  env_in_third_stage=model(model_request3)

  model_request4= chat_prompt.format_prompt(lifecycle_stage=obj.Fourth_stage, objects=name).to_messages()
  env_in_fourth_stage=model(model_request4)


#Printing the results
  result=f'''
  {name} definition: {definition.content}
  \n {name} usually has a specific life cycle and some environment in its lifecycle.
  \n The first stage of the life cycle of {name} is {obj.first_stage}. In this stage, the environment is classified as the following:\n{env_in_first_stage.content}
  \n The second stage of the life cycle of {name} is {obj.Second_stage}. In this stage, the environment is classified as the following:\n{env_in_second_stage.content}
  \n The third stage of the life cycle of {name} is {obj.Third_stage}. In this stage, the environment is classified as the following:\n{env_in_third_stage.content}
  \n The fourth stage of the life cycle of {name} is {obj.Fourth_stage}. In this stage, the environment is classified as the following:\n{env_in_fourth_stage.content}
  '''
  return print(result)

In [11]:
model = ChatOpenAI(temperature=0)

In [13]:
What_OR_Who_is('house')


  house definition: A house is a building that serves as a dwelling for individuals or families, providing shelter, security, and a sense of home.
  
 house usually has a specific life cycle and some environment in its lifecycle.
  
 The first stage of the life cycle of house is Design and Planning. In this stage, the environment is classified as the following:
1. The built environment of the house includes the layout, structure, and materials used in construction.
2. The human environment of the house involves the occupants, their needs, preferences, and daily activities within the space.
3. The nature environment of the house encompasses factors such as sunlight, wind direction, and proximity to natural elements like trees or bodies of water.
  
 The second stage of the life cycle of house is Construction. In this stage, the environment is classified as the following:
1. The built environment of the house is bustling with activity as construction workers lay the foundation and erect

**Third_stage: Identifying Interactions**

In [14]:
def active_verb_extraction(text):

  #Extracting action verbs
    system_template1='''An action verb, also known as a dynamic verb, is a type of verb that describes an action or something that a person or thing does (e.g., “I  run”).
    Action verbs differ from stative verbs, which describe a state of being (e.g., “believe,” “want”, "is", "are", "am", "have", "has", "become", "feel").
    Note: Gerund (verb +ing) are not action verb. While run is an action verb, runing is not an action verb.
    You must return nothing if there is no action verb in the sentence.
    As an AI research assistant who can analyze sentences, your task is to identify action verbs in a sentence'''
    system_message_prompt1= SystemMessagePromptTemplate.from_template(system_template1)
    human_template1='Please return the action verbs in this text:\n {text}'
    human_message_prompt1=HumanMessagePromptTemplate.from_template(human_template1)
    prompt1=ChatPromptTemplate.from_messages([system_message_prompt1,human_message_prompt1])
    chain1=LLMChain(llm=llm, prompt=prompt1, output_key='action_verbs')

#Splitting text to the sentences, each sentence has one action verb
    system_template2= '''As an AI assistant,You would get some action verbs.
    Your task is to identify these action verbs in the text and split the text to independent sentences.
    Each sentence has only one active verb.
    You must return nothing if there is no action verb in the sentence.
    Your output is only a sentence without extra explaination.
    '''
    system_message_prompt2= SystemMessagePromptTemplate.from_template(system_template2)
    human_template2='''Regard these action verbs:\n {action_verbs}.\n
    In the following text, identify these action verbs and return a sentence with this format: subject + action verb + object.
    This is the text:\n {text}'''
    human_message_prompt2=HumanMessagePromptTemplate.from_template(human_template2)
    prompt2=ChatPromptTemplate.from_messages([system_message_prompt2,human_message_prompt2])
    chain2=LLMChain(llm=llm, prompt=prompt2, output_key='active_sentence')

#Creating the chain
    seq_chain=SequentialChain(chains=[chain1,chain2], input_variables=['text'],output_variables=['action_verbs','active_sentence'],verbose=False)

#Since we use the period sign to end sentences, we can split the text into sentences using this sign
    sentences = text.split(".")
    active_sentences_text=''

# Inputting each sentence to the chain created earlier
    for l in sentences:
       out_put=seq_chain(l)
       active_sentences_text=active_sentences_text + f"{out_put['active_sentence']}\n"

    print(f"the first stage is: \n {active_sentences_text} \n END \n")





In [15]:
text='''
house definition: A house is a building that serves as a dwelling for individuals or families, providing shelter, security, and a sense of home.

 house usually has a specific life cycle and some environment in its lifecycle.

 The first stage of the life cycle of house is Design and Planning. In this stage, the environment is classified as the following:
1. The built environment of the house includes the layout, structure, and materials used in construction.
2. The human environment of the house involves the occupants, their needs, preferences, and daily activities within the space.
3. The nature environment of the house encompasses factors such as sunlight, wind direction, and proximity to natural elements like trees or bodies of water.

 The second stage of the life cycle of house is Construction. In this stage, the environment is classified as the following:
1. The built environment of the house is bustling with activity as construction workers lay the foundation and erect the walls.
2. The human environment of the house is filled with architects, engineers, and laborers working together to bring the design to life.
3. The nature environment surrounding the house is impacted by the construction, with trees being cleared and the landscape being altered to make way for the new structure.

 The third stage of the life cycle of house is Occupancy. In this stage, the environment is classified as the following:
1. The built environment of the house includes the walls, floors, and roof that provide shelter and protection.
2. The human environment of the house consists of the occupants who live, work, or visit the space.
3. The nature environment of the house encompasses the surrounding landscape, weather conditions, and natural elements that impact the building.

 The fourth stage of the life cycle of house is Maintenance and Renovation. In this stage, the environment is classified as the following:
1. The built environment of the house is undergoing maintenance and renovation, with workers repairing walls, replacing flooring, and updating fixtures.
2. The human environment of the house is bustling with activity as contractors, designers, and homeowners collaborate to improve the living space.
3. The nature environment surrounding the house is temporarily disrupted by construction noise, dust, and debris as the renovation progresses.

'''

In [16]:
active_verb_extraction(text)

the first stage is: 
 A house serves as a dwelling for individuals or families.
House has a specific life cycle.
The first stage of the life cycle of a house is Design.
The environment is classified.
The built environment of the house includes the layout, structure, and materials.
Nothing to return.
The human environment of the house involves the occupants.  
The human environment of the house needs their needs.  
The human environment of the house preferences.  
The human environment of the house activities within the space.
Nothing
The nature environment of the house encompasses factors.
Construction.
The environment is classified.
Construction workers lay the foundation.  
Construction workers erect the walls.
Nothing to return.
Architects, engineers, and laborers working together to bring the design to life.
Nothing
Construction impacted the nature environment.
Trees cleared.
Landscape altered.
Structure make way.
The third stage of the life cycle of house is Occupancy.
The environ

**Fourth Stage: Establishing Functional analysis**

In [17]:
# To create a functional analysis, we would utilize JSON to generate structured output.
json_schema2={'title':'Functional analysis',
             'description':'Identfying tools, objcets and their interactions',
             'type':'object',
             'properties':{
                 'Tool': {'title':'tool','description':'The objcet that change another object or protect another object to be changed','data_type':'string'},
                 'Object': {'title':'Object','description':'The Object which is changed or being protected form a change','data_type':'string'},
                 'How tool affect the object': {'title':'How tool affect on the object','description':'How tool affect on the object','data_type':'string'},
                 'How object affect the tool': {'title':'How object affect the tool','description':'How object affect the tool','data_type':'string'},
             },

             'required':['Tool','Object','How tool affect the object','How object affect the tool']
             }

In [18]:
def functional_analysis(text):
  #Defining the tools and objects for the LLM, and extracting the relationship among them from each interaction.
  system_template='''you are an AI assistant that could analyze a sentence and specify tool, object and the impact of tools on object and the impact of object on tool'''
  system_message_prompt= SystemMessagePromptTemplate.from_template(system_template)

  text1= 'Alex write a sentence'
  example_one= HumanMessagePromptTemplate.from_template(text1)
  text1_out_put= 'Tool: Alex, Object: Sentence, How tool affect the object: creating the sentence, How object affect the tool:None'
  example_one_output= AIMessagePromptTemplate.from_template(text1_out_put)

  text2='Ali eats a sandwich'
  example_two= HumanMessagePromptTemplate.from_template(text2)
  text2_out_put= 'Tool: Ali, Object: Sandwich,  How tool affect the object: consuming the sandwich, How object affect the tool:None'
  example_two_output= AIMessagePromptTemplate.from_template(text2_out_put)

  text3='Dog walks near the school'
  example_three= HumanMessagePromptTemplate.from_template(text3)
  text3_out_put= 'specified action means tool does something that impact the object. There is imapct on dog or school. Thus there is no tool, and object'
  example_three_output= AIMessagePromptTemplate.from_template(text3_out_put)

  text4='The manager interview a job applicant'
  example_four= HumanMessagePromptTemplate.from_template(text4)
  text4_out_put= 'Tool: Job applicant, Object: Manger,  How tool affect the object: increasing the information of manager, How object affect the tool:None '
  example_four_output= AIMessagePromptTemplate.from_template(text4_out_put)

  Human_prompt= '''In every action there is a tool and object. Action means tool does something that causes object to change or to be maintained from the change.
  Thus in every action there are two material object (one is tool and the other is object) that are interacting with each other.
  In This interaction the tool must impact the object. If there is no impact on the object, we can conclude that there is no action.
  The object that impact on the other one is called as a tool and the one that is got affected is the bject.
  Please consider these sentences:\n {text} \n please clarify waht are tools and objects in every sentence.
  Please also clarify how the tool affects the object and how the object affect the tool'''
  prompt= HumanMessagePromptTemplate.from_template(Human_prompt)
  chat_prompt=ChatPromptTemplate.from_messages([prompt,system_message_prompt])

  # Creating the structured out put with the json defined earlier
  chain=create_structured_output_chain(json_schema2,llm, chat_prompt,verbose=False)

  #To make it easier to use, we will input all the interactions into the function. Using the split() method, we will separate each sentence (interaction) and use LLM to consider each interaction separately.
  sentences = text.split(".")
  result=[]

#Executing the chain
  for l in sentences:
   out_put=chain.run(text=l)
   print(f"\n the sentence is: \n {l}")
   print(f" the functional analysis is: \n {out_put}")
   result.append(out_put)

In [19]:
text='''

 A house serves as a dwelling for individuals or families.
The human environment of the house involves the occupants.
The human environment of the house preferences.
The nature environment of the house encompasses factors.
The environment is classified.
Construction workers lay the foundation.
Construction workers erect the walls.
Architects, engineers, and laborers working together to bring the design to life.
Construction impacted the nature environment.
Trees cleared.
Landscape altered.
Structure make way.
The built environment of the house includes the walls.
The occupants live in the house.
The occupants work in the house.
The occupants visit the space.
The nature environment of the house encompasses the surrounding landscape.
Weather conditions impact the building.
Natural elements impact the building.
The environment is classified.
Workers repairing walls.
Workers replacing flooring.
Workers updating fixtures.
Contractors, designers, and homeowners collaborate to improve the living space.
The nature environment is disrupted.
The renovation progresses.
'''

In [20]:
functional_analysis(text)


 the sentence is: 
 

 A house serves as a dwelling for individuals or families
 the functional analysis is: 
 {'Tool': 'House', 'Object': 'Individuals or families', 'How tool affect the object': 'Provides shelter and living space', 'How object affect the tool': 'Occupies and utilizes the space'}

 the sentence is: 
 
The human environment of the house involves the occupants
 the functional analysis is: 
 {'Tool': 'human environment of the house', 'Object': 'occupants', 'How tool affect the object': 'provides a living space and amenities for the occupants', 'How object affect the tool': 'occupants utilize and interact with the human environment of the house'}

 the sentence is: 
   
The human environment of the house preferences
 the functional analysis is: 
 {'Tool': 'human environment', 'Object': 'house preferences', 'How tool affect the object': 'The human environment influences the house preferences by shaping the atmosphere, design, and functionality of the living space.', 'How o

**Fifth Stage: Idea Generation with TRIZ tools**

In [21]:
#So far, all the temperature settings for each section and function have been at 0 to produce consistent and predictable output. However, we have now increased the temperature setting to 1 to enhance the creativity of GPT and generate better ideas in terms of both quality and quantity.
llm_h= ChatOpenAI(temperature=1)

In [22]:
def TRIZ_principles(problem, contradiction, principles):

  #Asking LLm to generate ideas for each TRIZ principle
  system_template='''you are an AI assistant that could get a design problem and its contradiction and generate ideas regarding TRIZ principles.
  The out-put text should clearly state generated ideas and provide an example for each idea to better explain them.
  Your out-put is only the ideas and examples witout extra explaination.'''
  system_message_prompt= SystemMessagePromptTemplate.from_template(system_template)

  Human_prompt= '''TRIZ is an acronym for "Theory of Inventive Problem Solving."
   It is a problem-solving methodology that originated in Russia and was developed by Genrich Altshuller and his colleagues in the mid-20th century.
   The 40 Principles of TRIZ are a set of guidelines formulated by Genrich Altshuller, the founder of TRIZ.
   These principles are based on his analysis of thousands of patents across various industries.
   They serve as strategies for overcoming contradictions and generating inventive solutions for problem-solving and innovation.
   According to TRIZ in every design problem there is a contradiction that would make this design problme.
   Now, I would like to give you a design problem with its contradiction. Please resolve it using Principle {number} of TRIZ.
   \n Here's the problem:\n{problem}
   \n This is the contradiction recognized in the problem:\n {contradiction}.
   '''
  prompt= HumanMessagePromptTemplate.from_template(Human_prompt)
  chat_prompt=ChatPromptTemplate.from_messages([prompt,system_message_prompt])
  ideas=''
  Principle_number=principles
  for x in range(Principle_number):
    model_request= chat_prompt.format_prompt(number= str(x+1),problem=problem, contradiction=contradiction).to_messages()
    result=llm_h(model_request)
    print(f"\n{result.content}")

    #saving all the generated ideas in a list. This list usually includes more than 100 ideas.
    ideas= ideas + result.content



    # Splitting the list of ideas to shorter chunck and removing duplicated ideas in each chunk
    text_splitter=CharacterTextSplitter.from_tiktoken_encoder(chunk_size=600)
    splitted_ideas=text_splitter.split_text(ideas)



    system_template='''
   You are an AI assistant who is an expert in removing repetitive ideas.
   I will provide you with a list of ideas and their examples.
   Some of these ideas may be repetitive, so please remove any duplicates and return a list of unique ideas with their examples.
  '''
    system_message_prompt= SystemMessagePromptTemplate.from_template(system_template)
    human_template= 'Please consider the list of ideas and only return a list of unique ideas. \n list of ideas: \n {ideas} '
    human_message_prompt=HumanMessagePromptTemplate.from_template(human_template)
    prompt0=ChatPromptTemplate.from_messages([system_message_prompt,human_message_prompt])

    summerized_ideas= ''

    for k in splitted_ideas:
     request=prompt0.format_prompt(ideas=k).to_messages()
     summerized_ideas = summerized_ideas + llm(request).content




  #Removing duplicated ideas for the second time
    system_template='''
   You are an AI assistant who is an expert in removing repetitive ideas.
   I will provide you with a list of ideas and their examples.
   Some of these ideas may have repetitive concept, so please remove ideas with the same concepts and return a list of unique ideas with their examples.
   '''
    system_message_prompt= SystemMessagePromptTemplate.from_template(system_template)
    human_template= 'Please consider the list of ideas and only return a list of unique ideas. \n list of ideas: \n {summerized_ideas} '
    human_message_prompt=HumanMessagePromptTemplate.from_template(human_template)
    prompt0=ChatPromptTemplate.from_messages([system_message_prompt,human_message_prompt])

    request2=prompt0.format_prompt(summerized_ideas=summerized_ideas).to_messages()
    second_summerized_ideas=llm(request2).content



    # LLM is requested to consider the design problem, review the generated ideas, and suggest team of experts for more consultation.
    system_template='''
   You are an AI assistant who is an expert to detremine which experts are required in a design team for resolving a design problem.
   you get the design problem, contradiction in the design problem and generated ideas.
   You would return the experts needed to validate and implement the ideas to resolve the design problem and create a novel product.
   '''
    system_message_prompt= SystemMessagePromptTemplate.from_template(system_template)
    human_template= '''this is the design problem: \n {problem}
    \n\n this is the contradiction inside the design problem: {contradiction}
    \n\n this is the generated ideas:{final_ideas}
    '''
    human_message_prompt=HumanMessagePromptTemplate.from_template(human_template)
    prompt0=ChatPromptTemplate.from_messages([system_message_prompt,human_message_prompt])

    request3=prompt0.format_prompt(problem=problem, contradiction=contradiction, final_ideas=second_summerized_ideas).to_messages()
    design_team=llm(request3).content





#printing the results
  return print(f" \n \n This is the contradiction that we face:\n{contradiction}.\n \n \n Regarding TRIZ principles from number 1 to number {Principle_number}, we would have these ideas:\n\n{summerized_ideas} \n\n\n--------------\n\n\n second_summerized ideas:\n{second_summerized_ideas} \n\n\n These experts can help you: \n{design_team}")







In [23]:
problem='''Design a house that can fly'''
contradiction='''I want the material of the house to be light to reduce the house weight, and I want the material of the house to be not light because the light material is not sturdy,'''

In [24]:
TRIZ_principles(problem=problem, contradiction=contradiction, principles=15)


Ideas generated using Principle 1 of TRIZ:
  
 1. Segmentation: Divide the house into smaller segments for easier movement and reduced weight. For example, design the house to have detachable sections that can be assembled and disassembled when needed for flight.
   
 2. Porous Materials: Use materials with small holes or pores to reduce weight while maintaining strength. For example, incorporate lightweight but durable materials like aerogel or carbon fiber composites in the construction of the house.

Principle 2 of TRIZ: Taking Out
Ideas:
1. Use inflatable materials for the structure of the house, allowing it to be lightweight and easy to transport.
Example: Design the house with inflatable walls and roof that can be filled with helium gas for buoyancy when flying.
2. Implement a modular design for the house, so components can be easily assembled and disassembled for transportation, reducing overall weight.
Example: Create a house with modular sections that can be assembled on-site